# Generating Sentences One Letter at a Time
In this script we will create several simple character based language models using very simple conditional probability distributions (CPDs) by calculating simple histograms from the data.  

Specifically, we will use **Markov Models** that remember either 0, 1, 2, 3 or 4 previous characters and emit a probability distrubiton over the next character in the sequence.  
Markov Models are perhaps the simplest and most streight forward way to model temporal sequences of discrete symbols (in our case, we will use characters as our discrete symbols).  

We will see how discriminative these models can be for the task of author identification (spoiler: dispite their simplicity they are surprisingly good).  

We then continue to generate text in the style of the authors using those models to see what they have learned (another spoiler: they learn quite a bit, but not quite at the "famous author" level yet).   

Finally, we create a submission using the best model (4 character history)

**Note: **I use only pandas, numpy and native python here to make it simpler to "get into" the code for those of you who wish to do so.

![nice animation](https://cdn-images-1.medium.com/max/1600/1*MbHRwYNA8F29hzes8EPHiQ.gif)

Check out this [blog post](https://hackernoon.com/from-what-is-a-markov-model-to-here-is-how-markov-models-work-1ac5f4629b71) from which I stole this nice animation above. It explains the basics of markov models and even contains code to apply markov models at the word level as basic discrete symbols. In this script, however, we will use characters as the basic discrete symbols. 

# Short Math Introduction
In this script we try to model the distribution of sentences $$P(Sentence)$$  
A sentence is just a sequence of $n$ characters and therrefore we can write $$P(Sentence) = P(c_1,c_2,c_3,...,c_n)$$  

**In this script** we will model this sequence of characters using short term memory conditional distributions.  
* **no memory** (this is also known as naive bayes): 
$$P(c_1,c_2,c_3,...,c_n) = P(c_1)P(c_2)P(c_3)...P(c_n) = \prod_{t=1}^{n}P(c_t)$$ 
* **1 time step memory** (this is the classical markov chain): 
$$P(c_1,c_2,c_3,...,c_n) = P(c_1)P(c_2|c_1)P(c_3|c_2)...P(c_n|c_{n-1}) = P(c_1)\prod_{t=2}^{n}P(c_t|c_{t-1})$$ 
* **2 time step memory**: 
$$P(c_1,c_2,c_3,...,c_n) = P(c_1)P(c_2|c_1)\prod_{t=3}^{n}P(c_t|c_{t-1},c_{t-2})$$ 
* **3 time step memory**: 
$$P(c_1,c_2,c_3,...,c_n) = P(c_1)P(c_2|c_1)P(c_3|c_2,c_1)\prod_{t=4}^{n}P(c_t|c_{t-1},c_{t-2},c_{t-3})$$ 
* **4 time step memory**: 
$$P(c_1,c_2,c_3,...,c_n) = P(c_1)P(c_2|c_1)P(c_3|c_2,c_1)P(c_4|c_3,c_2,c_1)\prod_{t=5}^{n}P(c_t|c_{t-1},c_{t-2},c_{t-3},c_{t-4})$$ 

## Note: 
In this corpus, the authors use 34 total characters (regular characters plus punctuation marks).  
What this means is that:  
Storing the $P(c_t)$ distribution will involve storing $34$ numbers.  
Storing the $P(c_t|c_{t-1})$ distribution will involve storing $34^2 = 1,156$ numbers.  
Storing the $P(c_t|c_{t-1},c_{t-2})$ distribution will involve storing $34^3 = 39,304$ numbers.  
Storing the $P(c_t|c_{t-1},c_{t-2},c_{t-3})$ distribution will involve storing $34^4 = 1,336,336$ numbers.  
Storing the $P(c_t|c_{t-1},c_{t-2},c_{t-3},c_{t-4})$ distribution will involve storing $34^5 = 45,435,424$ numbers.  

Keep in mind also that the entire corpus of this competition contains about ~3M characters so that we should start encountering substantial finite sample size effects for 3 and 4 history conditional probability distributions.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn import model_selection, preprocessing
from sklearn.metrics import log_loss

matplotlib.style.use('fivethirtyeight')

## Load training data and seperate it into a train and validation sets

In [2]:
#%% load and organize data
data = pd.read_csv('../input/train.csv')

stratifiedCV = model_selection.StratifiedShuffleSplit(n_splits=1, test_size=0.10, random_state=1)
trainInds, validInds = next(stratifiedCV.split(data['text'], data['author']))

trainText  = data.loc[trainInds,'text'].reset_index(drop=True)
validText  = data.loc[validInds,'text'].reset_index(drop=True)
trainLabel = data.loc[trainInds,'author'].reset_index(drop=True)
validLabel = data.loc[validInds,'author'].reset_index(drop=True)

## Collect all chars into one large string for each author

In [3]:
#%% some utility code
# dictionary to manually converts greek/spanish chars into closest english chars
toEnglishDict = {}
srcStr = ['à','â','ä','å','æ','ç','è','é','ê','ë','ï','î','ñ','ô','ö','õ','ü','û','α','δ','ν','ο','π','ς','υ','ἶ']
dstStr = ['a','a','a','a','a','c','e','e','e','e','i','i','n','o','o','o','u','u','a','d','n','o','p','s','y','i']
for src,dst in zip(srcStr,dstStr):
    toEnglishDict[src] = dst
    
# function that converts all non english chars to their closest english char counterparts
def myunidecode(inString):
    outString = ''
    for ch in inString:
        if ch in toEnglishDict.keys():
            outString += toEnglishDict[ch]
        else:
            outString += ch
    return outString

In [4]:
#%% go over all train data and collect one large char sequence for each author
charsDict = {}
for key in ['all','EAP','HPL','MWS']:
    charsDict[key] = []

for k, (sentence, author) in enumerate(zip(trainText,trainLabel)):
    # the decoding is done for spanish/greek chars to be converted to close english chars
    decodedSentence = myunidecode(sentence.lower())
    chars = [char for char in decodedSentence]
    
    charsDict['all']  += chars
    charsDict[author] += chars

# Show the Char usage Distribution for each Author
$$P(c_t|Author)$$

In [5]:
#%% show char usage histogram for the three authors
charEncoder = preprocessing.LabelEncoder()
charEncoder.fit(charsDict['all'])

charCounts_EAP = np.histogram(charEncoder.transform(charsDict['EAP']),range(len(charEncoder.classes_)+1),density=True)[0]
charCounts_HPL = np.histogram(charEncoder.transform(charsDict['HPL']),range(len(charEncoder.classes_)+1),density=True)[0]
charCounts_MWS = np.histogram(charEncoder.transform(charsDict['MWS']),range(len(charEncoder.classes_)+1),density=True)[0]

# sort the char classes by their usage frequency
sortedChars = np.flipud(np.argsort(charCounts_EAP + charCounts_HPL + charCounts_MWS))

In [6]:
barWidth = 0.21
x = np.arange(len(charCounts_EAP))

plt.figure(figsize=(12,7)); plt.title('Character Usage Frequncy - $P(C_t)$ ',fontsize=25);
plt.bar(x-barWidth, charCounts_EAP[sortedChars], barWidth, color='r', label='Edgar Allen Poe');
plt.bar(x         , charCounts_HPL[sortedChars], barWidth, color='g', label='Howard Phillips Lovecraft');
plt.bar(x+barWidth, charCounts_MWS[sortedChars], barWidth, color='b', label='Mary Wollstonecraft Shelley');
plt.legend(fontsize=24); plt.ylabel('Usage Frequncy - $P(C_t)$', fontsize=20); plt.xlabel('$C_t$');
plt.xticks(x,["'%s'" %(charEncoder.classes_[i]) for i in sortedChars], fontsize=13);

Interestingly, there are differences between the authors here!  
## Lets look at the same plot only with log scale on the y axis

In [7]:
barWidth = 0.21
x = np.arange(len(charCounts_EAP))

plt.figure(figsize=(12,7)); 
plt.title('Character Usage Frequncy - $P(C_t)$ ',fontsize=25);
plt.bar(x-barWidth, charCounts_EAP[sortedChars], barWidth, color='r', label='Edgar Allen Poe');
plt.bar(x         , charCounts_HPL[sortedChars], barWidth, color='g', label='Howard Phillips Lovecraft');
plt.bar(x+barWidth, charCounts_MWS[sortedChars], barWidth, color='b', label='Mary Wollstonecraft Shelley');
plt.legend(fontsize=21); plt.ylabel('Usage Frequncy - $P(C_t)$', fontsize=20); 
plt.yscale("log", nonposy='clip'); plt.xlabel('$C_t$');
plt.xticks(x,["'%s'" %(charEncoder.classes_[i]) for i in sortedChars], fontsize=11);

Very large differences can be seen in usage of punctuation marks.  
For example, the ":" character is used much more extensivley by MWS and least used by HPL.

Even though these differences look small, they are extreemly statistically significant since there are about 2,600,000 chars in the training set. These differences are not there by chance, and perhaps we can utilize these differences for classification. Lets try.

# How do we Identify the Author of each Sentence using our CPDs?
In this competition specifically, we would like to model the conditional probability distribution of the author given a sentence: $$P(Author | Sentence) = P(Author | c_1,c_2,c_3,...,c_n)$$  

We have three different authors here, so this boils down to:  

$$P(EAP | c_1,c_2,c_3,...,c_n)$$    
$$P(HPC | c_1,c_2,c_3,...,c_n)$$    
$$P(MWS | c_1,c_2,c_3,...,c_n)$$  

Using the bayes rule we can invert this into $$P(Author | Sentence) = \frac{P(Sentence | Author)P(Author)}{P(Sentence)}$$  
The prior distribution $P(Author)$ over authors is extreemly simple to calcualte (it's just the frequency of occurence of each author in the training set).  

For any specific sentence, the probability of that sentence $P(Sentence)$ is identical for all authors and therefore doesn't create any additional discrimination between them, so when creating a classifier we can just ignore this and compare the 3 following quantities:  

$$P(c_1,c_2,c_3,...,c_n | EAP)P(EAP)$$    
$$P(c_1,c_2,c_3,...,c_n | HPC)P(HPC)$$    
$$P(c_1,c_2,c_3,...,c_n | MWS)P(MWS)$$  
meaning:
$$  \mathbf{predicted\:Author} = argmax\:\{{P(c_1,c_2,c_3,...,c_n|Author)P(Author)}\}   $$

# Calculate Classification Accuracy based only on single char distribution
$$  \mathbf{predicted\:Author} = argmax\:\{ \prod_{t=1}^{n}P(c_t|Author)\}   $$  

**Note:** we assume here an equal prior just for simplicity (i.e. $P(Author) = \frac{1}{3}$ for all authors)  


In [8]:
#%% meassure classification accuracy on validation set using only character frequncy
authorsList = ['EAP','HPL','MWS']
authorPredictionList = []
for k, (sentence, author) in enumerate(zip(validText,validLabel)):
    chars = [char for char in myunidecode(sentence.lower())]
    # convert to log so we can sum probabilities instead of multiply
    logP_EAP = sum([np.log(charCounts_EAP[charEncoder.classes_ == ch]) for ch in chars])
    logP_HPL = sum([np.log(charCounts_HPL[charEncoder.classes_ == ch]) for ch in chars])
    logP_MWS = sum([np.log(charCounts_MWS[charEncoder.classes_ == ch]) for ch in chars])
    
    authorPredictionList.append(authorsList[np.argmax(np.array([logP_EAP,logP_HPL,logP_MWS]))])

print(52*'-')
print('==> Validation Set Classification Accuracy = %.1f%s' %(100*(validLabel == authorPredictionList).mean(),'%'))
print(52*'-')

----------------------------------------------------
==> Validation Set Classification Accuracy = 53.0%
----------------------------------------------------


Interestingly, even though this is perhaps the stupidest model one can think of, the discrimination accuracy is well above chance level.

## Generate Sample Text for each author using the independent chars model
$$  c_t \: {\raise.17ex\hbox{$\scriptstyle\mathtt{\sim}$}} \: P(c_t|Author)   $$

In [9]:
#%% generate sample text by sampling one charachter at a time for the independent character model
np.random.seed(1234)

maxSentenceLength = 95
numSentencesPerAuthor = 5

charProbModel = {}
charProbModel['all'] = (charCounts_EAP + charCounts_HPL + charCounts_MWS)/3.0
charProbModel['EAP'] = charCounts_EAP
charProbModel['HPL'] = charCounts_HPL
charProbModel['MWS'] = charCounts_MWS

for author in ['EAP','HPL','MWS','all']:
    print((6+maxSentenceLength)*'-')
    print('Author %s:' %(author))
    print(12*'-')
    for i in range(numSentencesPerAuthor):
        generatedSentence = ''
        for j in range(maxSentenceLength):
            newChar = charEncoder.classes_[np.random.choice(range(len(charCounts_EAP)),size=1,p=charProbModel[author])][0]
            generatedSentence += newChar
            
            if (newChar == '.') or (j == maxSentenceLength):
                break
                
        print('%d: "%s"' %(i+1,generatedSentence))
print((4+maxSentenceLength)*'-')

-----------------------------------------------------------------------------------------------------
Author EAP:
------------
1: "."
2: "mfrrbcrvtehnoeih rtem etnerditfr ooatft , nli idh li mtryvrcmh."
3: "e fw  olh atei dfmtmo osnf iiuhhis nroriv  l ud."
4: "gttae,tootrlc en  dlsc" y,di twi eo ibwna rrptniaoutysiftwledtaoerromhsof eohen  o el ruwtmtoo "
5: "fce phnmefehxe ttemeaae npteceste   l c m sdpi in,rmimnarsoohsneie ksun e "i"ihtotcaotxndlsm cy"
-----------------------------------------------------------------------------------------------------
Author HPL:
------------
1: "ekia hyeo,   cirtnsloit r    hez dyetiutr viata owif lvwt."
2: "mee tn trdsh ffasolhhblpbi otiy, gtshtadoo clte hodstrbuuotahruntn seuh v n  nwope aq h,miisuao"
3: "oatiideewank ia t wde  em w,phrd go nelabts ntoh  s it nh   a e ttr."
4: "ltw donmsoe qe ot ,ee  f st afwr tse   t enlee eorat ta dsseaic mi pyreeeflm,to s n ae frnii ly"
5: "alsrdevaa hbi egroa ae hstay er fesoe oeskutg  dllh toes  ht i   tf nh

Well, as expected, we can see that this really doesn't resemble any human generated text.  
But perhaps we can do better with some memory?

## Gather DataFrame with "Author", "History" and "Next Char" Fields
Use history of 1 character 

In [10]:
#%% gather all pairs of characters into a single dataframe
historyLength = 1

historyList  = []
nextCharList = []
authorList   = []
for k, (sentence, author) in enumerate(zip(trainText,trainLabel)):
    decodedSentence = myunidecode(sentence.lower())
    charSequences = [decodedSentence[k:k+historyLength+1] for k in range(len(decodedSentence)-historyLength)]
    
    history  = [seq[:-1] for seq in charSequences]
    nextChar = [seq[ -1] for seq in charSequences]
    sentenceDF = pd.DataFrame(columns=['author','history','next char'])
    
    historyList  += history
    nextCharList += nextChar
    authorList   += [author]*len(history)
        
corpusDF = pd.DataFrame(columns=['author','history','next char'])
corpusDF['author']    = authorList
corpusDF['history']   = historyList
corpusDF['next char'] = nextCharList

corpusDF.head(8)

,author,history,next char
0,EAP,t,h
1,EAP,h,e
2,EAP,e,
3,EAP,,w
4,EAP,w,e
5,EAP,e,a
6,EAP,a,t
7,EAP,t,h


## Build Markov Model that Remebers only the previous char
$$P(c_t|c_{t-1},Author)$$

In [11]:
#%% generate P(c(t)|c(t-1)) model (Markov Model with memory of 1 time step)
charCondProbModel_H1 = {}
for author in ['EAP','HPL','MWS']:
    charCondProbModel_H1[author] = np.zeros( (1+historyLength)*[charEncoder.classes_.shape[0]] )
    
charCondCountModel_H1 = {}
for author in ['EAP','HPL','MWS']:
    charCondCountModel_H1[author] = np.zeros( (1+historyLength)*[charEncoder.classes_.shape[0]] )

corpusGroupedByAuthor = corpusDF.groupby(by='author',axis=0)
for author in corpusDF['author'].unique():
    authorCorpusDF = corpusGroupedByAuthor.get_group(author).loc[:,['history','next char']].reset_index(drop=True)
    authorCorpusGroupedByHistory = authorCorpusDF.groupby(by='history',axis=0)
    for history in authorCorpusDF['history'].unique():
        authorHistoryDF = authorCorpusGroupedByHistory.get_group(history).reset_index(drop=True).loc[:,'next char'].reset_index(drop=True)

        encodedHistory = charEncoder.transform([history])[0]
        encodedNextCharCounts = np.histogram(charEncoder.transform(authorHistoryDF),range(len(charEncoder.classes_)+1),density=False)[0]
        encodedNextCharProb   = np.histogram(charEncoder.transform(authorHistoryDF),range(len(charEncoder.classes_)+1),density=True)[0]

        charCondProbModel_H1[author][encodedHistory,:]  = encodedNextCharProb
        charCondCountModel_H1[author][encodedHistory,:] = encodedNextCharCounts

    condCount = charCondCountModel_H1[author]
    print('%s Sparsity level = %.1f%s' %(author, 100*(condCount < 1).sum() / (condCount > -1).sum().astype(float),'%'))

charCondProbModel_H1['all']  = (charCondProbModel_H1['EAP']  + charCondProbModel_H1['HPL']  + charCondProbModel_H1['MWS'] )/3.0
charCondCountModel_H1['all'] =  charCondCountModel_H1['EAP'] + charCondCountModel_H1['HPL'] + charCondCountModel_H1['MWS']

print('average Sparsity level = %.1f%s' %(100*(charCondCountModel_H1['all'] < 1).sum() / (condCount > -1).sum().astype(float),'%'))

EAP Sparsity level = 30.4%
MWS Sparsity level = 38.6%
HPL Sparsity level = 33.9%
average Sparsity level = 25.1%


# Show the Conditional Probability Distribution of the entire corpus 
$$P(c_t|c_{t-1}) $$

In [12]:
condProb = charCondProbModel_H1['all']

plt.figure(figsize=(12,10))
plt.imshow(condProb, cmap='hot');  plt.colorbar(); plt.clim(0,1);
plt.grid('off'); plt.title('P(next char | prev char) for all Authors - $P(c_t|c_{t-1})$', fontsize=22);
plt.xlabel('$c_t$ - next character', fontsize=18); plt.ylabel('$c_{t-1}$ - previous character', fontsize=18);
plt.xticks(range(condProb.shape[0]),["'%s'" %(ch) for ch in charEncoder.classes_]);
plt.yticks(range(condProb.shape[0]),["'%s'" %(ch) for ch in charEncoder.classes_]);

There are a few clear "columns" in the dataset - for the "a","e","i" and "o" characters (which are vowels, by the way). What this means is that these characters are quite likely to come after many other characters. Contrast that with "y" that is likely to occur mostly after "l", "m" and "b" (to form the pairs "ly", "my" and "by").

Note that we now have about 25% of character pairs that never occur in the training set at all. Even though we only have ~1,150 possible charachter pairs and ~2,600,000 pairs.  
In this particular case, it's safe to assume that the pairs that don't occur are simply very rare or non existent in the languge, but it's important to keep in mind that some of these zeros are perhaps due to finite sample size. This can be quanitfied of course, but we will skip this in this tutorial.

# Show the Author Specific Conditional Probability Distributions 
$$P(c_t|c_{t-1},Author) $$

In [13]:
shortToFullNameDict = {}
shortToFullNameDict['EAP'] = 'Edgar Allen Poe'
shortToFullNameDict['HPL'] = 'Howard Phillips Lovecraft'
shortToFullNameDict['MWS'] = 'Mary Wollstonecraft Shelley'

plt.figure(figsize=(13,28))
for k, author in enumerate(['EAP','HPL','MWS']):
    condProb = charCondProbModel_H1[author]
    plt.subplot(3,1,k+1); plt.imshow(condProb, cmap='hot'); 
    plt.grid('off'); plt.colorbar(); plt.clim(0,1);
    plt.title('P(next char | prev char, %s) - $P(c_t|c_{t-1},Author)$' %(shortToFullNameDict[author]), fontsize=17);
    plt.xlabel('$c_t$ - next character', fontsize=15); plt.ylabel('$c_{t-1}$ - previous character', fontsize=15);
    plt.xticks(range(condProb.shape[0]),["'%s'" %(ch) for ch in charEncoder.classes_]);
    plt.yticks(range(condProb.shape[0]),["'%s'" %(ch) for ch in charEncoder.classes_]);
plt.tight_layout();

We can see a few differences between the authors, especially in the top few rows that indicate the different usage of punctuation marks by our authors.

## Calculate Classification Accuracy of a Classic Markov Model
$$  \mathbf{predicted\:Author} = argmax\:\{ \prod_{t=2}^{n}P(c_t|c_{t-1},Author)\}   $$

In [14]:
#%% meassure classification accuracy on validation set using Markov Model with memory of 1 time step
uniformPriorFraction    = 0.0001
allAuthorsPriorFraction = 0.0001

prior = np.array([1.0-uniformPriorFraction-allAuthorsPriorFraction, allAuthorsPriorFraction, uniformPriorFraction])
uniformPriorValue = 1.0/len(charEncoder.classes_)

condP_H1 = {}
authorsList = ['EAP','HPL','MWS']
for author in authorsList:
    condP_H1[author]  = prior[0]*charCondProbModel_H1[author]
    condP_H1[author] += prior[1]*charCondProbModel_H1['all']
    condP_H1[author] += prior[2]*uniformPriorValue

authorPredictionList = []
for k, (sentence, author) in enumerate(zip(validText,validLabel)):
    decodedSentence = myunidecode(sentence.lower())
    charSequences = [decodedSentence[k:k+historyLength+1] for k in range(len(decodedSentence)-historyLength)]
    
    history  = [seq[:-1] for seq in charSequences]
    nextChar = [seq[ -1] for seq in charSequences]
    
    logP_EAP = 0.0; logP_HPL = 0.0; logP_MWS = 0.0
    for histChar, nextChar in zip(history,nextChar):
        encodedHistChar = charEncoder.transform([histChar])[0]
        encodedNextChar = charEncoder.transform([nextChar])[0]
        
        logP_EAP += np.log(condP_H1['EAP'][encodedHistChar,encodedNextChar])
        logP_HPL += np.log(condP_H1['HPL'][encodedHistChar,encodedNextChar])
        logP_MWS += np.log(condP_H1['MWS'][encodedHistChar,encodedNextChar])
    print(logP_EAP, logP_HPL, logP_MWS)
    
    authorPredictionList.append(authorsList[np.argmax([logP_EAP,logP_HPL,logP_MWS])])
    
print(52*'-')
print('==> Validation Set Classification Accuracy = %.1f%s' %(100*(validLabel == authorPredictionList).mean(),'%'))
print(52*'-')

-85.7398113486 -85.7136320293 -87.0094178698
-771.897084101 -770.045850232 -763.122823318
-373.801311856 -378.277095704 -378.870554064
-344.701787304 -351.815340002 -331.892958859
-729.745085623 -729.142948604 -720.727632415
-148.16269637 -150.897450574 -148.411006078
-150.337157077 -150.864742296 -150.13377326
-94.8625010346 -93.4672771825 -94.7509170561
-531.632634585 -516.052960687 -529.553971691
-371.011607429 -378.422598154 -374.770588025
-367.657263727 -364.058512992 -360.360353839
-181.469669781 -178.477198871 -181.019961473
-542.427350305 -536.829799991 -539.501692935
-723.633952671 -723.032748967 -722.574485801
-625.036220927 -617.615549485 -625.782863731
-529.941995087 -523.579154554 -526.878786612
-463.658029962 -458.924212411 -459.106476507
-493.235656039 -492.918894185 -492.496667399
-70.7883230312 -73.2117953019 -70.941967687
-434.108005569 -436.563003585 -433.718040179
-415.128884438 -413.942729124 -411.49533622
-926.25870527 -916.796209843 -923.026162231
-220.046134653 

-409.060995644 -407.047961687 -412.034912176
-767.265488967 -764.840576934 -764.514406219
-719.544625353 -729.848668289 -725.821867656
-399.606486149 -400.166246914 -401.340091278
-210.895746181 -206.867901015 -210.82100108
-239.176137609 -237.083663222 -241.88740091
-136.734819944 -136.138620544 -135.293897799
-772.569421083 -785.531994322 -767.955685331
-473.515331007 -472.238693453 -478.866146017
-636.462021213 -626.692118014 -631.705624868
-266.713725823 -265.26262358 -269.667542012
-202.278133996 -202.2476701 -198.90830968
-254.712873249 -261.36333156 -252.957730345
-231.266932928 -227.063947091 -229.056980097
-332.747358053 -330.791331457 -337.574240722
-808.284920186 -804.903882977 -812.730396129
-540.236009358 -535.451564713 -537.929002087
-765.853850282 -764.465285798 -771.457364916
-727.336223449 -731.009691868 -734.271893097
-395.303017216 -398.276388886 -399.561313091
-205.148917628 -204.862585069 -208.543029747
-396.4648543 -389.108535995 -388.87575919
-195.673169346 -197.

-669.656409348 -677.368586228 -662.115124678
-144.821485601 -144.599412278 -145.911322521
-251.006055539 -256.054175554 -253.617208744
-562.393751781 -559.756042425 -558.298280241
-612.407400989 -609.846464814 -613.319165155
-378.544992717 -379.469706313 -382.668207287
-447.051167829 -444.116379858 -440.975715316
-413.092726048 -407.407367214 -410.519790392
-144.180898237 -143.63959774 -141.805349381
-736.743555248 -737.637497821 -740.1916633
-184.569719349 -185.762839715 -183.331065143
-164.002388727 -165.791972267 -165.349744642
-377.134062098 -368.51196774 -375.606186867
-83.1822933479 -85.2960805494 -82.495139404
-204.819180018 -204.033110747 -202.315132336
-340.492332718 -339.291912265 -343.396498854
-633.942064389 -633.534430328 -633.696772485
-562.865460281 -568.46478738 -565.913673661
-292.743747656 -292.837998145 -289.048111018
-324.783731181 -321.987454326 -320.070273939
-524.823760371 -525.928153572 -521.847169698
-336.982482084 -335.072806922 -339.063690719
-573.665480397 -

-709.286399119 -711.797936431 -713.63067114
-438.009442314 -433.570135363 -427.263755568
-462.807371002 -466.3004242 -464.594617858
-293.172954042 -292.288300721 -290.850078709
-429.338750646 -434.594869446 -428.106271443
-1235.60808392 -1241.26052898 -1291.79370393
-259.53097173 -255.943848923 -258.04156144
-637.652930503 -630.117144326 -640.938072689
-415.504629773 -419.017919405 -412.123482026
-362.793850633 -355.49157148 -359.348064308
-288.310433053 -290.053083934 -291.791355394
-749.124713545 -740.334380574 -755.435211671
-1138.37331347 -1139.86301602 -1138.36777861
-452.106943194 -453.117859017 -454.048954601
-496.420316401 -494.475359055 -508.715189169
-415.174354567 -417.397723485 -418.848456919
-598.739518795 -593.699703218 -593.293349461
-412.893526918 -418.09850359 -426.036152676
-73.4829150821 -75.7651822966 -75.5251352024
-253.637957678 -253.008674944 -254.165000845
-571.229198478 -579.703816587 -577.228811715
-339.187056134 -338.706560369 -339.466045287
-777.854153844 -7

-162.410742475 -162.096696221 -161.055463413
-667.658566229 -678.530579047 -672.855703915
-81.2111050857 -81.795340366 -82.8144902895
-281.33207052 -283.760897892 -281.661116145
-110.911415526 -111.830456369 -111.673040795
-498.504705409 -494.501777225 -500.769100093
-890.475509012 -888.862082261 -897.967341035
-104.550336362 -103.62224215 -104.550413202
-57.8364462393 -57.5067823175 -57.6504828748
-335.142100838 -345.039904132 -332.265316352
-430.256000111 -427.541618084 -422.587947889
-387.280480376 -386.969563134 -386.051057299
-301.497649765 -304.535822033 -313.640434915
-590.85671754 -585.322677988 -582.379027592
-448.589479353 -454.35641334 -453.257289442
-485.28432624 -492.914361148 -476.715050928
-148.556498332 -149.471774423 -148.701056198
-1062.70371262 -1066.18462647 -1048.78104376
-667.750195116 -665.394067176 -666.200160694
-197.369943438 -197.291455381 -196.379955482
-404.23319228 -399.992518834 -403.152983258
-381.598151799 -382.453912058 -383.715444089
-213.968465546 -2

...

-327.363531402 -330.887007262 -333.773236569
-606.510784934 -609.377626018 -612.971615271
-418.47043825 -427.192076205 -422.93317932
-96.7308388097 -97.453282532 -97.5631628354
-1141.98876989 -1140.04159861 -1150.99128577
-203.6422677 -201.042027423 -198.502399825
-941.331503362 -955.173692561 -951.338732831
-428.970890457 -428.919490841 -426.047567621
-137.052069894 -135.889580981 -135.901423578
-298.461151491 -298.12806628 -303.172896408
-358.289104392 -362.574571454 -360.87970459
-146.304829215 -145.884827568 -147.906696171
-988.199674888 -979.804129074 -978.277478536
-101.88394684 -103.83124202 -102.794854927
-226.277204574 -228.645719863 -226.707131595
-557.761081499 -560.10260241 -555.026591697
-941.953773866 -941.053949769 -946.741906864
-258.559426328 -261.755904385 -258.52744287
-227.080236449 -224.194614336 -225.286921942
-173.132595998 -171.724726559 -174.916374823
-74.2709164696 -74.6933500881 -74.5191164368
-469.492177247 -465.05131373 -467.598688007
-280.128120195 -276.23

-138.795710244 -128.339040041 -145.231914822
-182.804740103 -180.55599639 -182.71544714
-149.778776664 -148.500331369 -146.978519556
-294.762089592 -295.939502801 -299.492484672
-367.523786013 -368.645522368 -365.553279241
-650.601832242 -649.400133157 -652.959935289
-304.858127715 -302.795496343 -302.673243671
-586.035353046 -583.396499467 -585.912494492
-288.850005674 -288.447201196 -286.696193009
-109.637746036 -111.171903509 -110.81875822
-351.297903532 -354.375572143 -350.865928741
-599.158575205 -616.083248975 -614.24672288
-368.095185565 -367.823679206 -365.467261917
-133.162134699 -133.683304675 -130.968058593
-1727.45732 -1698.5795169 -1713.70573754
-381.241418557 -378.589850129 -379.439714799
-245.885820619 -251.517907468 -253.869059937
-382.312441997 -376.690312252 -380.062106738
-523.693129638 -526.816575566 -522.729539958
-146.123282353 -147.691274806 -147.349771507
-268.494118094 -265.832005488 -265.899996982
-328.286239396 -324.159572304 -321.885243676
-176.150791956 -17

-154.429835414 -157.970338348 -158.317072636
-180.902590788 -181.183580635 -176.577830117
-341.166858833 -342.61355861 -338.909746569
-684.522619134 -682.021963278 -679.269787357
-286.669986336 -283.283371177 -286.417503199
-455.965482031 -473.238513105 -482.157285147
-355.663704238 -344.325880511 -373.885580618
-443.740953463 -445.569828239 -440.959464145
-264.945518374 -265.67015195 -267.59864318
-491.477356872 -490.483110084 -497.638334393
-702.601172345 -698.976824778 -700.830686884
-140.369124927 -139.879059794 -141.018460041
-774.164481616 -765.950936007 -776.078436265
-216.214733928 -213.471439576 -216.929731205
-1140.54159126 -1146.84908674 -1146.65028959
-372.665749799 -370.65942161 -379.809501709
-311.275797894 -311.220320341 -307.272460345
-56.9151871842 -57.7440666724 -56.152794305
-500.829367171 -504.078842495 -505.187918542
-270.246331578 -276.394658625 -271.140686351
-253.934337403 -259.436580589 -260.197666434
-192.380507803 -189.928417733 -190.843632178
-1627.62493052 

-465.196234476 -461.093794404 -465.043931963
-725.72774281 -724.078961472 -726.064097454
-311.77025019 -310.626727755 -308.782257158
-359.061264019 -359.849702693 -355.602769533
-759.204711459 -763.253312799 -756.363051608
-1216.89538665 -1216.96046995 -1209.07027361
-215.433848836 -212.113378422 -218.788062698
-553.840079425 -567.62669058 -566.852641215
-490.473834345 -491.401905836 -487.465876346
-323.96194705 -326.648363178 -325.94697754
-721.95389788 -725.947194196 -730.099154627
-351.826595894 -351.481849616 -349.407565302
-125.911281233 -130.771864657 -128.969062035
-319.256545734 -318.97509364 -316.944832342
-773.813668274 -770.372773702 -772.92918087
-173.634787472 -173.952313471 -176.699353542
-343.715327067 -345.777792376 -345.176719915
-127.903623568 -128.279733011 -126.61570404
-185.371704995 -189.643169225 -183.570200528
-271.163942576 -272.027196187 -267.376438483
-175.589356444 -165.872423413 -179.290629508
-187.153139312 -187.541952761 -186.150536542
-433.119740677 -430

-657.938030519 -656.032050098 -663.743898466
-87.1679303584 -88.2709873877 -85.0319015083
-331.269898516 -334.910232945 -327.041814512
-249.085787725 -247.043817306 -243.72467612
-470.108554347 -470.32641862 -472.552097347
-204.247485895 -208.207640613 -206.249396312
-280.949084978 -277.870598486 -274.020615555
-403.714967175 -399.181174648 -404.273916376
-386.97023907 -390.24197704 -384.071181214
-151.266697759 -151.352933444 -150.298667187
-811.39793968 -804.239683045 -814.876371683
-270.031005786 -274.260752814 -265.006020214
-1103.14945452 -1106.11945168 -1098.56170663
-192.48348206 -193.936540262 -192.338137586
-646.960777961 -644.438334112 -647.540698938
-117.952526348 -117.594441958 -117.099706796
-471.513313567 -466.429396979 -467.861343425
-238.920555764 -239.652721468 -235.600636297
-161.818882498 -167.087099292 -161.670963248
-208.094226949 -207.402965286 -206.946432067
-243.722350242 -245.807615938 -242.447721583
-155.273254913 -160.078879646 -159.862528446
-276.953037503 -

The classification accuracy increases some more. By just looking at the distribution of pairs of charachters. 

## Generate Sample Text for each Author using our Markov Model
$$  c_t\: {\raise.17ex\hbox{$\scriptstyle\mathtt{\sim}$}} \:  P(c_t|c_{t-1},Author)   $$

In [15]:
#%% generate sample text by sampling one charachter at a time from the 1 time step memory Markov Model
np.random.seed(123)

maxSentenceLength = 90
numSentencesPerAuthor = 6

uniformPriorFraction    = 0.0001
allAuthorsPriorFraction = 0.0009

prior = np.array([1.0-uniformPriorFraction-allAuthorsPriorFraction, allAuthorsPriorFraction, uniformPriorFraction])
uniformPriorValue = 1.0/(len(charEncoder.classes_))

condP_H1 = {}
authorsList = ['EAP','HPL','MWS']
for author in authorsList:
    condP_H1[author]  = prior[0]*charCondProbModel_H1[author]
    condP_H1[author] += prior[1]*charCondProbModel_H1['all']
    condP_H1[author] += prior[2]*uniformPriorValue

condP_H1['all']  = (prior[0]+prior[1])*charCondProbModel_H1['all']
condP_H1['all'] += prior[2]*uniformPriorValue

for author in ['EAP','HPL','MWS','all']:
    print((6+maxSentenceLength)*'-')
    print('Author %s:' %(author))
    print(12*'-')
    for i in range(numSentencesPerAuthor):
        firstChar = charEncoder.classes_[np.random.choice(range(len(charCounts_EAP)),size=1,p=charProbModel[author])][0]
        generatedSentence = firstChar
        for j in range(maxSentenceLength-1):
            encodedHistChar = charEncoder.transform([generatedSentence[-1]])[0]
            newChar = charEncoder.classes_[np.random.choice(range(len(charCounts_EAP)),size=1,p=condP_H1[author][encodedHistChar,:])][0]
            generatedSentence += newChar
            
            if (newChar == '.') or (j == maxSentenceLength):
                break
        print('%d: "%s"' %(i+1,generatedSentence))
print((4+maxSentenceLength)*'-')

------------------------------------------------------------------------------------------------
Author EAP:
------------
1: "of orexple, i, s cingoris, d pand imesthe hes my o, oncecounghens tafe the on brdegatol ga"
2: "oxcoor igang bensere'e tefee wof mutapreparsubjed, freder st te r rsondeibotespos toud bin"
3: "icerer oupende p se n py."
4: "jeng thered hiecuits lim e ert gupo my anterof, ieves fa y ie d pea he ofi t bt sutitholy "
5: "ust o s aiteeifenandrgthe my andi veshimeas."
6: "autenag."
------------------------------------------------------------------------------------------------
Author HPL:
------------
1: "dlorve alackeg s fooliourasquseraly thanor amy f d dinctes sed aly he bese wilysare ald cu"
2: " pheata vigh."
3: " thet; artheesifod ofomaz, as aror falockyof wnthieat the verste."
4: "lethemey d bere wh re osld ngasi our, t t nd qusese bthe, wesitowisorng mofand ndan arnton"
5: "me h, cht betr wintomind ld wendy cauth deseal, wid rouguinewheer , mbererer me won ced

This already has a text like feeling to it. After punctioation we see a whitespace, every now and then we see the letter "a" and "i" seperated by whitespaces and the short word "he" appears several times in the text. we are getting somewhere, let's add a little bit more memory.

## Gather DataFrame with "Author", "History" and "Next Char" Fields
Use history of size 2 characters

In [16]:
#%% gather all triplets of characters into a single dataframe
historyLength = 2

historyList  = []
nextCharList = []
authorList   = []
for k, (sentence, author) in enumerate(zip(trainText,trainLabel)):
    decodedSentence = myunidecode(sentence.lower())
    charSequences = [decodedSentence[k:k+historyLength+1] for k in range(len(decodedSentence)-historyLength)]
    
    history  = [seq[:-1] for seq in charSequences]
    nextChar = [seq[ -1] for seq in charSequences]
    sentenceDF = pd.DataFrame(columns=['author','history','next char'])
    
    historyList  += history
    nextCharList += nextChar
    authorList   += [author]*len(history)
        
corpusDF = pd.DataFrame(columns=['author','history','next char'])
corpusDF['author']    = authorList
corpusDF['history']   = historyList
corpusDF['next char'] = nextCharList

corpusDF.head(8)

,author,history,next char
0,EAP,th,e
1,EAP,he,
2,EAP,e,w
3,EAP,w,e
4,EAP,we,a
5,EAP,ea,t
6,EAP,at,h
7,EAP,th,e


## Build Markov Model that remebers the Two previous chars
$$ P(c_t|c_{t-1},c_{t-2},Author)  $$

In [17]:
#%% generate P(c(t)|c(t-1),c(t-2)) model (Markov Model with memory of 2 time steps)
historyLength = 2

charCondProbModel_H2 = {}
for author in ['EAP','HPL','MWS']:
    charCondProbModel_H2[author] = np.zeros( (1+historyLength)*[charEncoder.classes_.shape[0]] )
    
charCondCountModel_H2 = {}
for author in ['EAP','HPL','MWS']:
    charCondCountModel_H2[author] = np.zeros( (1+historyLength)*[charEncoder.classes_.shape[0]] )

corpusGroupedByAuthor = corpusDF.groupby(by='author',axis=0)
for author in corpusDF['author'].unique():
    authorCorpusDF = corpusGroupedByAuthor.get_group(author).loc[:,['history','next char']].reset_index(drop=True)
    authorCorpusGroupedByHistory = authorCorpusDF.groupby(by='history',axis=0)
    for history in authorCorpusDF['history'].unique():
        authorHistoryDF = authorCorpusGroupedByHistory.get_group(history).reset_index(drop=True).loc[:,'next char'].reset_index(drop=True)

        encodedHistory = charEncoder.transform([ch for ch in history])
        encodedNextCharCounts = np.histogram(charEncoder.transform(authorHistoryDF),range(len(charEncoder.classes_)+1),density=False)[0]
        encodedNextCharProb   = np.histogram(charEncoder.transform(authorHistoryDF),range(len(charEncoder.classes_)+1),density=True)[0]

        charCondProbModel_H2[author][encodedHistory[0],encodedHistory[1],:]  = encodedNextCharProb
        charCondCountModel_H2[author][encodedHistory[0],encodedHistory[1],:] = encodedNextCharCounts

    condCount = charCondCountModel_H2[author]
    print('%s Sparsity level = %.1f%s' %(author, 100*(condCount < 1).sum() / (condCount > -1).sum().astype(float),'%'))

charCondProbModel_H2['all']  = (charCondProbModel_H2['EAP']  + charCondProbModel_H2['HPL']  + charCondProbModel_H2['MWS'] )/3.0
charCondCountModel_H2['all'] =  charCondCountModel_H2['EAP'] + charCondCountModel_H2['HPL'] + charCondCountModel_H2['MWS']

condCount = charCondCountModel_H2['all']
print('average Sparsity level = %.1f%s' %(100*(condCount < 1).sum() / (condCount > -1).sum().astype(float),'%'))

EAP Sparsity level = 81.5%
MWS Sparsity level = 85.0%
HPL Sparsity level = 82.8%
average Sparsity level = 78.2%


Note the Sparsity Levels increase quite a bit. This is due to the fact that there are 34^3 (~40,000) possible triplets, and most of them are illegal combinations in the english language.  

But remember what we discussed earlier, here the problem of finite sample size is much more pronounced so we can be much less "confident" in these zeros. meaning, we can't be sure they are actually zeros and not simply very rare events that just didn't happen to occur in the particular realization of the training sample.    

For this reason we will add a small constant number to the conditional probability distribution.

## Calculate Classification Accuracy of our 2 time step Markov Model
$$  \mathbf{predicted\:Author} = argmax\:\{ \prod_{t=3}^{n}P(c_t|c_{t-1},c_{t-2},Author)\}   $$

In [18]:
#%% meassure classification accuracy on validation set using Markov Model with memory of 2 time steps
uniformPriorFraction    = 0.0001
allAuthorsPriorFraction = 0.0001

prior = np.array([1.0-uniformPriorFraction-allAuthorsPriorFraction, allAuthorsPriorFraction, uniformPriorFraction])
uniformPriorValue = 1.0/len(charEncoder.classes_)

condP_H2 = {}
authorsList = ['EAP','HPL','MWS']
for author in authorsList:
    condP_H2[author]  = prior[0]*charCondProbModel_H2[author]
    condP_H2[author] += prior[1]*charCondProbModel_H2['all']
    condP_H2[author] += prior[2]*uniformPriorValue

authorPredictionList = []
for k, (sentence, author) in enumerate(zip(validText,validLabel)):
    decodedSentence = myunidecode(sentence.lower())
    charSequences = [decodedSentence[k:k+historyLength+1] for k in range(len(decodedSentence)-historyLength)]
    
    history  = [seq[:-1] for seq in charSequences]
    nextChar = [seq[ -1] for seq in charSequences]
    
    logP_EAP = 0.0; logP_HPL = 0.0; logP_MWS = 0.0
    for histChars, nextChar in zip(history,nextChar):
        encodedHistChars = charEncoder.transform([ch for ch in histChars])
        encodedNextChar  = charEncoder.transform([nextChar])[0]
        
        logP_EAP += np.log(condP_H2['EAP'][encodedHistChars[0],encodedHistChars[1],encodedNextChar])
        logP_HPL += np.log(condP_H2['HPL'][encodedHistChars[0],encodedHistChars[1],encodedNextChar])
        logP_MWS += np.log(condP_H2['MWS'][encodedHistChars[0],encodedHistChars[1],encodedNextChar])
    
    authorPredictionList.append(authorsList[np.argmax([logP_EAP,logP_HPL,logP_MWS])])

print(52*'-')
print('==> Validation Set Classification Accuracy = %.1f%s' %(100*(validLabel == authorPredictionList).mean(),'%'))
print(52*'-')

----------------------------------------------------
==> Validation Set Classification Accuracy = 74.8%
----------------------------------------------------


The Classification Accuracy keeps rising, which is quite nice.  
We are now quite capable in distinguishing between the three authors.  
***Does this also mean that our model has the ability to write text like our authors?***

## Generate Sample Text for each Author using our 2 time step Markov Model
$$  c_t\: {\raise.17ex\hbox{$\scriptstyle\mathtt{\sim}$}} \:  P(c_t|c_{t-1},c_{t-2},Author)   $$

In [19]:
#%% generate sample text by sampling one charachter at a time from the 2 time step Markov Model
np.random.seed(1000)

maxSentenceLength = 95
numSentencesPerAuthor = 9

uniformPriorFraction    = 0.0001
allAuthorsPriorFraction = 0.0009

prior = np.array([1.0-uniformPriorFraction-allAuthorsPriorFraction, allAuthorsPriorFraction, uniformPriorFraction])
uniformPriorValue = 1.0/(len(charEncoder.classes_))

condP_H2 = {}
authorsList = ['EAP','HPL','MWS']
for author in authorsList:
    condP_H2[author]  = prior[0]*charCondProbModel_H2[author]
    condP_H2[author] += prior[1]*charCondProbModel_H2['all']
    condP_H2[author] += prior[2]*uniformPriorValue

condP_H2['all']  = (prior[0]+prior[1])*charCondProbModel_H2['all']
condP_H2['all'] += prior[2]*uniformPriorValue

for author in ['EAP','HPL','MWS','all']:
    print((6+maxSentenceLength)*'-')
    print('Author %s:' %(author))
    print(12*'-')
    for i in range(numSentencesPerAuthor):
        firstChar = charEncoder.classes_[np.random.choice(range(len(charCounts_EAP)),size=1,p=charProbModel[author])][0]
        encodedFirstChar = charEncoder.transform([firstChar])[0]
        secondChar = charEncoder.classes_[np.random.choice(range(len(charCounts_EAP)),size=1,p=condP_H1[author][encodedFirstChar,:])][0]
        generatedSentence = firstChar + secondChar
        
        for j in range(maxSentenceLength-1):
            encodedHistChars = charEncoder.transform([ch for ch in generatedSentence[-2:]])            
            currCondProb = condP_H2[author][encodedHistChars[0],encodedHistChars[1],:]
            currCondProb = currCondProb/currCondProb.sum() # just in case the probabilities don't sum directly to 1
            newChar = charEncoder.classes_[np.random.choice(range(len(charCounts_EAP)),size=1,p=currCondProb)][0]
            generatedSentence += newChar
            
            if (newChar == '.') or (j == maxSentenceLength):
                break
        print('%d: "%s"' %(i+1,generatedSentence))
print((4+maxSentenceLength)*'-')

-----------------------------------------------------------------------------------------------------
Author EAP:
------------
1: "n wholanclencen torb youl the to accons kage to and seend ded not orraing, hint itir, anceperess"
2: "nd a vation the mis at the eposed."
3: "s briver no min ted was prout batup as dis alre scalang a hateduct, watterut onge walver a pent "
4: "and le suff thad the do douloncess, aus wit to ex, ex pon brable ne pery younes is of was fuld d"
5: "chimpert shis joh ant mance his so wo thesecits, able dat was was on a lores i had thady by the "
6: "appres priould men to stre wild not in yourpas un the wor the but hat anow andess, ithantly, ate"
7: "nd ren war mor partmos, ples grecippautim yetent beeked thattlive ridespred felf the of th he th"
8: "the mace hat trywha the beep," thated alich ah, of my de had, "th whosyche th the ond istruit in"
9: "d ever imes."
-----------------------------------------------------------------------------------------------------

Clearly the answer is no.  Our model is not as capable as our authors.  
Nevertheless, we see that the text looks even better now. A lot of short 2-4 letter words like  "on", "to", "we", "me", "of", "the", "for", "now", "age", "hate", "thin", "eyes" appear quite often.   
A completely different world relative to the independent model we saw first that looked like a complete jumble.

# Let's Repeat the process with History size of 3 chars
## Gather DataFrame with "Author", "History" and "Next Char" Fields
Use history of size 3 characters

In [20]:
#%% gather all quadruplets of characters into a single dataframe
historyLength = 3

historyList  = []
nextCharList = []
authorList   = []
for k, (sentence, author) in enumerate(zip(trainText,trainLabel)):
    decodedSentence = myunidecode(sentence.lower())
    charSequences = [decodedSentence[k:k+historyLength+1] for k in range(len(decodedSentence)-historyLength)]
    
    history  = [seq[:-1] for seq in charSequences]
    nextChar = [seq[ -1] for seq in charSequences]
    sentenceDF = pd.DataFrame(columns=['author','history','next char'])
    
    historyList  += history
    nextCharList += nextChar
    authorList   += [author]*len(history)
        
corpusDF = pd.DataFrame(columns=['author','history','next char'])
corpusDF['author']    = authorList
corpusDF['history']   = historyList
corpusDF['next char'] = nextCharList

corpusDF.head(8)

,author,history,next char
0,EAP,the,
1,EAP,he,w
2,EAP,e w,e
3,EAP,we,a
4,EAP,wea,t
5,EAP,eat,h
6,EAP,ath,e
7,EAP,the,r


## Build Markov Model that remebers the 3 previous chars
$$ P(c_t|c_{t-1},c_{t-2},c_{t-3},Author)  $$

In [21]:
#%% generate P(c(t)|c(t-1),c(t-2),c(t-3)) model (Markov Model with memory of 3 time steps)
historyLength = 3

charCondProbModel_H3 = {}
for author in ['EAP','HPL','MWS']:
    charCondProbModel_H3[author] = np.zeros( (1+historyLength)*[charEncoder.classes_.shape[0]] )
    
charCondCountModel_H3 = {}
for author in ['EAP','HPL','MWS']:
    charCondCountModel_H3[author] = np.zeros( (1+historyLength)*[charEncoder.classes_.shape[0]] )

corpusGroupedByAuthor = corpusDF.groupby(by='author',axis=0)
for author in corpusDF['author'].unique():
    authorCorpusDF = corpusGroupedByAuthor.get_group(author).loc[:,['history','next char']].reset_index(drop=True)
    authorCorpusGroupedByHistory = authorCorpusDF.groupby(by='history',axis=0)
    for history in authorCorpusDF['history'].unique():
        authorHistoryDF = authorCorpusGroupedByHistory.get_group(history).reset_index(drop=True).loc[:,'next char'].reset_index(drop=True)

        encodedHistory = charEncoder.transform([ch for ch in history])
        encodedNextCharCounts = np.histogram(charEncoder.transform(authorHistoryDF),range(len(charEncoder.classes_)+1),density=False)[0]
        encodedNextCharProb   = np.histogram(charEncoder.transform(authorHistoryDF),range(len(charEncoder.classes_)+1),density=True)[0]

        charCondProbModel_H3[author][encodedHistory[0],encodedHistory[1],encodedHistory[2],:]  = encodedNextCharProb
        charCondCountModel_H3[author][encodedHistory[0],encodedHistory[1],encodedHistory[2],:] = encodedNextCharCounts

    condCount = charCondCountModel_H3[author]
    print('%s Sparsity level = %.1f%s' %(author, 100*(condCount < 1).sum() / (condCount > -1).sum().astype(float),'%'))

charCondProbModel_H3['all']  = (charCondProbModel_H3['EAP']  + charCondProbModel_H3['HPL']  + charCondProbModel_H3['MWS'] )/3.0
charCondCountModel_H3['all'] =  charCondCountModel_H3['EAP'] + charCondCountModel_H3['HPL'] + charCondCountModel_H3['MWS']

condCount = charCondCountModel_H3['all']
print('average Sparsity level = %.1f%s' %(100*(condCount < 1).sum() / (condCount > -1).sum().astype(float),'%'))

EAP Sparsity level = 97.5%
MWS Sparsity level = 98.1%
HPL Sparsity level = 97.8%
average Sparsity level = 96.8%


Note the sparsity is increasing, as one would expect.

## Calculate Classification Accuracy of Markov Model that remebers 3 time steps back
$$  \mathbf{predicted\:Author} = argmax\:\{ \prod_{t=4}^{n}P(c_t|c_{t-1},c_{t-2},c_{t-3},Author)\}   $$

In [22]:
#%% meassure classification accuracy on validation set using Markov Model with memory of 3 time steps
uniformPriorFraction    = 0.05
allAuthorsPriorFraction = 0.05

prior = np.array([1.0-uniformPriorFraction-allAuthorsPriorFraction, allAuthorsPriorFraction, uniformPriorFraction])
uniformPriorValue = 1.0/(len(charEncoder.classes_))

condP_H3 = {}
authorsList = ['EAP','HPL','MWS']
for author in authorsList:
    condP_H3[author]  = prior[0]*charCondProbModel_H3[author]
    condP_H3[author] += prior[1]*charCondProbModel_H3['all']
    condP_H3[author] += prior[2]*uniformPriorValue

condP_H3['all']  = (prior[0]+prior[1])*charCondProbModel_H3['all']
condP_H3['all'] += prior[2]*uniformPriorValue

authorPredictionList = []
for k, (sentence, author) in enumerate(zip(validText,validLabel)):
    decodedSentence = myunidecode(sentence.lower())
    charSequences = [decodedSentence[k:k+historyLength+1] for k in range(len(decodedSentence)-historyLength)]
    
    history  = [seq[:-1] for seq in charSequences]
    nextChar = [seq[ -1] for seq in charSequences]
    
    logP_EAP = 0.0; logP_HPL = 0.0; logP_MWS = 0.0
    for histChars, nextChar in zip(history,nextChar):
        encodedHistChars = charEncoder.transform([ch for ch in histChars])
        encodedNextChar  = charEncoder.transform([nextChar])[0]
        
        logP_EAP += np.log(condP_H3['EAP'][encodedHistChars[0],encodedHistChars[1],encodedHistChars[2],encodedNextChar])
        logP_HPL += np.log(condP_H3['HPL'][encodedHistChars[0],encodedHistChars[1],encodedHistChars[2],encodedNextChar])
        logP_MWS += np.log(condP_H3['MWS'][encodedHistChars[0],encodedHistChars[1],encodedHistChars[2],encodedNextChar])
    
    authorPredictionList.append(authorsList[np.argmax([logP_EAP,logP_HPL,logP_MWS])])

print(52*'-')
print('==> Validation Set Classification Accuracy = %.1f%s' %(100*(validLabel == authorPredictionList).mean(),'%'))
print(52*'-')

----------------------------------------------------
==> Validation Set Classification Accuracy = 82.9%
----------------------------------------------------


The accuracy is already quite high!

## Generate Sample Text for each Author using our 3 time step Markov Model
$$  c_t\: {\raise.17ex\hbox{$\scriptstyle\mathtt{\sim}$}} \:  P(c_t|c_{t-1},c_{t-2},c_{t-3},Author)   $$

In [23]:
#%% generate sample text by sampling one charachter at a time from the 3 time step Markov Model
np.random.seed(123)

maxSentenceLength = 95
numSentencesPerAuthor = 9

uniformPriorFraction    = 0.05
allAuthorsPriorFraction = 0.05

prior = np.array([1.0-uniformPriorFraction-allAuthorsPriorFraction, allAuthorsPriorFraction, uniformPriorFraction])
uniformPriorValue = 1.0/(len(charEncoder.classes_))

condP_H3 = {}
authorsList = ['EAP','HPL','MWS']
for author in authorsList:
    condP_H3[author]  = prior[0]*charCondProbModel_H3[author]
    condP_H3[author] += prior[1]*charCondProbModel_H3['all']
    condP_H3[author] += prior[2]*uniformPriorValue

condP_H3['all']  = (prior[0]+prior[1])*charCondProbModel_H3['all']
condP_H3['all'] += prior[2]*uniformPriorValue

for author in ['EAP','HPL','MWS','all']:
    print((6+maxSentenceLength)*'-')
    print('Author %s:' %(author))
    print(12*'-')
    for i in range(numSentencesPerAuthor):
        # sample c(1) ~ P(c(t))
        firstChar  = charEncoder.classes_[np.random.choice(range(len(charCounts_EAP)),size=1,p=charProbModel[author])][0]
        encodedFirstChar = charEncoder.transform([firstChar])[0]
        # sample c(2) ~ P(c(t)|c(t-1))
        secondChar = charEncoder.classes_[np.random.choice(range(len(charCounts_EAP)),size=1,p=condP_H1[author][encodedFirstChar,:])][0]
        encodedSecondChar = charEncoder.transform([secondChar])[0]
        # sample c(3) ~ P(c(t)|c(t-1),c(t-2))
        thirdChar  = charEncoder.classes_[np.random.choice(range(len(charCounts_EAP)),size=1,p=condP_H2[author][encodedFirstChar,encodedSecondChar,:])][0]
        generatedSentence = firstChar + secondChar + thirdChar
        
        for j in range(maxSentenceLength-1):
            encodedHistChars = charEncoder.transform([ch for ch in generatedSentence[-historyLength:]])            
            currCondProb = condP_H3[author][encodedHistChars[0],encodedHistChars[1],encodedHistChars[2],:]
            currCondProb = currCondProb/currCondProb.sum() # just in case the probabilities don't sum directly to 1
            
            # sample c(t) ~ P(c(t)|c(t-1),c(t-2),c(t-3))
            newChar = charEncoder.classes_[np.random.choice(range(len(charCounts_EAP)),size=1,p=currCondProb)][0]
            generatedSentence += newChar
            
            if (newChar == '.') or (j == maxSentenceLength):
                break
        print('%d: "%s"' %(i+1,generatedSentence))
print((4+maxSentenceLength)*'-')

-----------------------------------------------------------------------------------------------------
Author EAP:
------------
1: "of stitude."
2: "on my consion; and all, buted agan it ration of rijnough escrip of continute enty repare no depre"
3: "s. get?" " tragglooking out king guinitely shed to know operselever a line the musion."
4: " pass of you shamber, and it wondeed absuresse'nse my stanrf;yov teave back."
5: "ch in shoctable,gvqygvex toj."
6: "l s:'?vekly perhjlykj ix, foughtly, equents of into ver gening seen his, and the fungb;ito, gream"
7: "ancifiquicken absolunadqpcythould faile was inteauvail ther ; stilliner companzas that said they "
8: "a would himself."
9: "isput of."
-----------------------------------------------------------------------------------------------------
Author HPL:
------------
1: "ed ivide hot what coundark, jerminer the everefn"."
2: "real;cgwoversitroubtle kill so andness circhas bewicz have pfabrowly xv;,yyn drophed wilding was "
3: "which "yes 

Note the large number of legal english words in the generated text.  
Our probabalistic model has managed to learn a lot of english words.  

Also note the relativley **long** 6+ letter words the model generates, such as "**exceeded**", "**remain**", "**expect**" "**danger**" and "**struct**", this while our model only remebers directly 4 character sequences, it manages to concatenate several such sequences together to form a longer coherent sequence at least some of the time.

# Let's Repeat the process one last time with History size of 4 chars
## Gather DataFrame with "Author", "History" and "Next Char" Fields
Use history of size 4 characters

In [24]:
#%% gather all 5-wise of characters into a single dataframe
historyLength = 4

historyList  = []
nextCharList = []
authorList   = []
for k, (sentence, author) in enumerate(zip(trainText,trainLabel)):
    decodedSentence = myunidecode(sentence.lower())
    charSequences = [decodedSentence[k:k+historyLength+1] for k in range(len(decodedSentence)-historyLength)]
    
    history  = [seq[:-1] for seq in charSequences]
    nextChar = [seq[ -1] for seq in charSequences]
    sentenceDF = pd.DataFrame(columns=['author','history','next char'])
    
    historyList  += history
    nextCharList += nextChar
    authorList   += [author]*len(history)
        
corpusDF = pd.DataFrame(columns=['author','history','next char'])
corpusDF['author']    = authorList
corpusDF['history']   = historyList
corpusDF['next char'] = nextCharList

corpusDF.head(15)

,author,history,next char
0,EAP,the,w
1,EAP,he w,e
2,EAP,e we,a
3,EAP,wea,t
4,EAP,weat,h
5,EAP,eath,e
6,EAP,athe,r
7,EAP,ther,
8,EAP,her,w
9,EAP,er w,a


## Build Markov Model that remebers the 4 previous chars
$$ P(c_t|c_{t-1},c_{t-2},c_{t-3},c_{t-4},Author)  $$

In [25]:
#%% generate P(c(t)|c(t-1),c(t-2),c(t-3),c(t-4)) model (Markov Model with memory of 4 time steps)
historyLength = 4

charCondProbModel_H4 = {}
for author in ['EAP','HPL','MWS']:
    charCondProbModel_H4[author] = np.zeros( (1+historyLength)*[charEncoder.classes_.shape[0]] )
    
charCondCountModel_H4 = {}
for author in ['EAP','HPL','MWS']:
    charCondCountModel_H4[author] = np.zeros( (1+historyLength)*[charEncoder.classes_.shape[0]] )

corpusGroupedByAuthor = corpusDF.groupby(by='author',axis=0)
for author in corpusDF['author'].unique():
    authorCorpusDF = corpusGroupedByAuthor.get_group(author).loc[:,['history','next char']].reset_index(drop=True)
    authorCorpusGroupedByHistory = authorCorpusDF.groupby(by='history',axis=0)
    for history in authorCorpusDF['history'].unique():
        authorHistoryDF = authorCorpusGroupedByHistory.get_group(history).reset_index(drop=True).loc[:,'next char'].reset_index(drop=True)

        encodedHistory = charEncoder.transform([ch for ch in history])
        encodedNextCharCounts = np.histogram(charEncoder.transform(authorHistoryDF),range(len(charEncoder.classes_)+1),density=False)[0]
        encodedNextCharProb   = np.histogram(charEncoder.transform(authorHistoryDF),range(len(charEncoder.classes_)+1),density=True)[0]

        charCondProbModel_H4[author][encodedHistory[0],encodedHistory[1],encodedHistory[2],encodedHistory[3],:]  = encodedNextCharProb
        charCondCountModel_H4[author][encodedHistory[0],encodedHistory[1],encodedHistory[2],encodedHistory[3],:] = encodedNextCharCounts

    condCount = charCondCountModel_H4[author]
    print('%s Sparsity level = %.2f%s' %(author, 100*(condCount < 1).sum() / (condCount > -1).sum().astype(float),'%'))

charCondProbModel_H4['all']  = (charCondProbModel_H4['EAP']  + charCondProbModel_H4['HPL']  + charCondProbModel_H4['MWS'] )/3.0
charCondCountModel_H4['all'] =  charCondCountModel_H4['EAP'] + charCondCountModel_H4['HPL'] + charCondCountModel_H4['MWS']

condCount = charCondCountModel_H4['all']
print('average Sparsity level = %.2f%s' %(100*((condCount < 1).sum() / (condCount > -1).sum().astype(float)),'%'))

EAP Sparsity level = 99.80%
MWS Sparsity level = 99.84%
HPL Sparsity level = 99.81%
average Sparsity level = 99.70%


## Calculate Classification Accuracy of Markov Model that remebers 4 time steps back
$$  \mathbf{predicted\:Author} = argmax\:\{ \prod_{t=5}^{n}P(c_t|c_{t-1},c_{t-2},c_{t-3},c_{t-4},Author)P(Author)\}   $$  
**Note:** I've added also the prior over authors, to give a small additional performance boost

In [26]:
#%% meassure classification accuracy on validation set using Markov Model with memory of 4 time steps
condP_H4_PriorWeight_specific = 70
condP_H4_PriorWeight_all      = 30

condP_H3_PriorWeight_specific = 70
condP_H3_PriorWeight_all      = 30

uniformPriorWeight            = 10

logP_EAP_prior = np.log((trainLabel == 'EAP').mean())
logP_HPL_prior = np.log((trainLabel == 'HPL').mean())
logP_MWS_prior = np.log((trainLabel == 'MWS').mean())

numChars = len(charEncoder.classes_)
prior = np.array([condP_H4_PriorWeight_specific, condP_H4_PriorWeight_all, 
                  condP_H3_PriorWeight_specific, condP_H3_PriorWeight_all, uniformPriorWeight])
prior = prior.astype(float) / prior.sum()

uniformPriorValue = 1.0/numChars

condP_H4 = {}
authorsList = ['EAP','HPL','MWS']
for author in authorsList:
    # get P(c(t)|c(t-1),c(t-2),c(t-3),c(t-4))
    condP_H4[author]  = prior[0]*charCondProbModel_H4[author]
    condP_H4[author] += prior[1]*charCondProbModel_H4['all']
    
    # get "prior" from P(c(t)|c(t-1),c(t-2),c(t-3))
    condP_H4_from_CondP_H3_specific = np.tile(charCondProbModel_H3[author][np.newaxis,:,:,:],[numChars,1,1,1,1])
    condP_H4_from_CondP_H3_all      = np.tile(charCondProbModel_H3['all'][np.newaxis,:,:,:],[numChars,1,1,1,1])
    condP_H4[author] += prior[2]*condP_H4_from_CondP_H3_specific
    condP_H4[author] += prior[3]*condP_H4_from_CondP_H3_all

    condP_H4[author] += prior[4]*uniformPriorValue

condP_H4['all']  = (condP_H4['EAP'] + condP_H4['HPL'] + condP_H4['MWS'])  / 3.0

authorPredictionList = []
logProbGivenAuthor = np.zeros((len(validLabel),3))
for i, (sentence, author) in enumerate(zip(validText,validLabel)):
    decodedSentence = myunidecode(sentence.lower())
    charSequences = [decodedSentence[k:k+historyLength+1] for k in range(len(decodedSentence)-historyLength)]
    
    history  = [seq[:-1] for seq in charSequences]
    nextChar = [seq[ -1] for seq in charSequences]
    
    logP_EAP = logP_EAP_prior; logP_HPL = logP_HPL_prior; logP_MWS = logP_MWS_prior;
    for histChars, nextChar in zip(history,nextChar):
        encodedHistChars = charEncoder.transform([ch for ch in histChars])
        encodedNextChar  = charEncoder.transform([nextChar])[0]
        
        logP_EAP += np.log(condP_H4['EAP'][encodedHistChars[0],encodedHistChars[1],encodedHistChars[2],encodedHistChars[3],encodedNextChar])
        logP_HPL += np.log(condP_H4['HPL'][encodedHistChars[0],encodedHistChars[1],encodedHistChars[2],encodedHistChars[3],encodedNextChar])
        logP_MWS += np.log(condP_H4['MWS'][encodedHistChars[0],encodedHistChars[1],encodedHistChars[2],encodedHistChars[3],encodedNextChar])
        
        logProbGivenAuthor[i,:] = [logP_EAP,logP_HPL,logP_MWS]
        
    authorPredictionList.append(authorsList[np.argmax([logP_EAP,logP_HPL,logP_MWS])])

print(52*'-')
print('==> Validation Set Classification Accuracy = %.1f%s' %(100*(validLabel == authorPredictionList).mean(),'%'))
print(52*'-')

----------------------------------------------------
==> Validation Set Classification Accuracy = 85.2%
----------------------------------------------------


The identification accuracy has reached quite a high level now.   

But we do see a saturation effect here. The improvment from history of 3 characters to 4 characters is not as large as the improvment from 2 character history to 3 character history.

## Let's calculate also the log loss
In order to relate this to LB results

In [27]:
#%% calculate log loss
minimalLogP = -15.0
uniformPriorWeight = 0.09

authorLogProb_norm = logProbGivenAuthor - np.tile(logProbGivenAuthor.max(axis=1)[:,np.newaxis], [1,3])
authorLogProb_norm[authorLogProb_norm < minimalLogP] = minimalLogP

authorProb = np.exp(authorLogProb_norm)
authorProb_norm = authorProb / np.tile(authorProb.sum(axis=1)[:,np.newaxis],[1, 3])

y_Hat = uniformPriorWeight*(1/3.0) + (1.0-uniformPriorWeight)*authorProb_norm

labelEncoder = preprocessing.LabelEncoder()
y_GT = labelEncoder.fit_transform(validLabel)

print(34*'-')
print('Validation Set Log Loss = %.5f' %(log_loss(y_GT, y_Hat)))
print(34*'-')

----------------------------------
Validation Set Log Loss = 0.42320
----------------------------------


## Generate Sample Text for each Author using our 4 time step Markov Model
$$  c_t\: {\raise.17ex\hbox{$\scriptstyle\mathtt{\sim}$}} \:  P(c_t|c_{t-1},c_{t-2},c_{t-3},c_{t-4},Author)   $$  
**Just for fun**, let's start all sentences with 'disp' and see how they evolve from there

In [28]:
np.random.seed(1000)

maxSentenceLength = 95
numSentencesPerAuthor = 9

condP_H4_PriorWeight_specific = 10
condP_H4_PriorWeight_all      = 10

condP_H3_PriorWeight_specific = 1
condP_H3_PriorWeight_all      = 1

uniformPriorWeight            = 1

numChars = len(charEncoder.classes_)
prior = np.array([condP_H4_PriorWeight_specific, condP_H4_PriorWeight_all, 
                  condP_H3_PriorWeight_specific, condP_H3_PriorWeight_all, uniformPriorWeight])
prior = prior.astype(float) / prior.sum()

uniformPriorValue = 1.0/numChars

condP_H4 = {}
authorsList = ['EAP','HPL','MWS']
for author in authorsList:
    # get P(c(t)|c(t-1),c(t-2),c(t-3),c(t-4))
    condP_H4[author]  = prior[0]*charCondProbModel_H4[author]
    condP_H4[author] += prior[1]*charCondProbModel_H4['all']
    
    # get prior from P(c(t)|c(t-1),c(t-2),c(t-3))
    condP_H4_from_CondP_H3_specific = np.tile(charCondProbModel_H3[author][np.newaxis,:,:,:],[numChars,1,1,1,1])
    condP_H4_from_CondP_H3_all      = np.tile(charCondProbModel_H3['all'][np.newaxis,:,:,:],[numChars,1,1,1,1])
    condP_H4[author] += prior[2]*condP_H4_from_CondP_H3_specific
    condP_H4[author] += prior[3]*condP_H4_from_CondP_H3_all

    condP_H4[author] += prior[4]*uniformPriorValue

condP_H4['all']  = (condP_H4['EAP'] + condP_H4['HPL'] + condP_H4['MWS'])  / 3.0

for author in ['EAP','HPL','MWS','all']:
    print((6+maxSentenceLength)*'-')
    print('Author %s:' %(author))
    print(12*'-')
    for i in range(numSentencesPerAuthor):
        generatedSentence = 'disp'
        for j in range(maxSentenceLength-1):
            encodedHistChars = charEncoder.transform([ch for ch in generatedSentence[-historyLength:]])            
            currCondProb = condP_H4[author][encodedHistChars[0],encodedHistChars[1],encodedHistChars[2],encodedHistChars[3],:]
            currCondProb = currCondProb/currCondProb.sum() # just in case the probabilities don't sum exactly to 1
            
            # sample c(t) ~ P(c(t)|c(t-1),c(t-2),c(t-3))
            newChar = charEncoder.classes_[np.random.choice(range(len(charCounts_EAP)),size=1,p=currCondProb)][0]
            generatedSentence += newChar
            
            if (newChar == '.') or (j == maxSentenceLength):
                break
        print('%d: "%s"' %(i+1,generatedSentence))
print((4+maxSentenceLength)*'-')

-----------------------------------------------------------------------------------------------------
Author EAP:
------------
1: "disposnive, and i says: "yes, the tu as thing so to admit of changelicating still undulum dream."
2: "disposition at the corpse that the enlighted sound no me frames less, was tte."
3: "dispered that carabaus."
4: "displace to you hq,vydxwz."
5: "disposit wakj;u "'oppq;aibjhd rted than the do its seems of what?" she was t,okd'."
6: "dispoke oar as to you must in tv."
7: "disposes no, northin the he cally jest despanies, theus other delic may divering a longed by a sma"
8: "dispath would distinctly une, and were, as before in you:",vcwb viness as that, both orded from di"
9: "disproperly."
-----------------------------------------------------------------------------------------------------
Author HPL:
------------
1: "dispellen."
2: "disposed we and eventury heart is nekf"yyub,z."
3: "disposite biologue out was self was dered in this birch were came as tryy.

Here we can almost see sentences:  
"...and i says: "yes, the..."  
"...when i spoken to put to think till my feel the also..."   
"...displayed on the done dark happines atter..."  
"..."why, so much as countain, and sent..."

Since we really know exactly what the model entails and it's simplicity, i.e. just storing conditional probabilities, and we see the generative performance of this model, that can remember english words and almost construct something that looks like actual sentences, this might make some of us wonder "could it be that our brains are just a somewhat better probability estimation machine?"  
My answer to this question would be "most likely yes" :-)

# Create a Submission on the Test Set
This submission might be useful for an ensemble if you haven't used any char based models yet

In [29]:
#%% create a submission
# load test data
testData = pd.read_csv('../input/test.csv')
testText = testData.loc[:,'text'].reset_index(drop=True)

# calculate log prob predictions
logProbGivenAuthor = np.zeros((len(testText),3))
for i, sentence in enumerate(testText):
    decodedSentence = myunidecode(sentence.lower())
    charSequences = [decodedSentence[k:k+historyLength+1] for k in range(len(decodedSentence)-historyLength)]
    
    history  = [seq[:-1] for seq in charSequences]
    nextChar = [seq[ -1] for seq in charSequences]
    
    logP_EAP = logP_EAP_prior; logP_HPL = logP_HPL_prior; logP_MWS = logP_MWS_prior;
    for histChars, nextChar in zip(history,nextChar):
        encodedHistChars = charEncoder.transform([ch for ch in histChars])
        encodedNextChar  = charEncoder.transform([nextChar])[0]
        
        logP_EAP += np.log(condP_H4['EAP'][encodedHistChars[0],encodedHistChars[1],encodedHistChars[2],encodedHistChars[3],encodedNextChar])
        logP_HPL += np.log(condP_H4['HPL'][encodedHistChars[0],encodedHistChars[1],encodedHistChars[2],encodedHistChars[3],encodedNextChar])
        logP_MWS += np.log(condP_H4['MWS'][encodedHistChars[0],encodedHistChars[1],encodedHistChars[2],encodedHistChars[3],encodedNextChar])
    
        logProbGivenAuthor[i,:] = [logP_EAP,logP_HPL,logP_MWS]

In [30]:
# convert log probabilities to final predictions
minimalLogP = -15.0
uniformPriorWeight = 0.09

authorLogProb_norm = logProbGivenAuthor - np.tile(logProbGivenAuthor.max(axis=1)[:,np.newaxis], [1,3])
authorLogProb_norm[authorLogProb_norm < minimalLogP] = minimalLogP
authorProb = np.exp(authorLogProb_norm)
authorProb_norm = authorProb / np.tile(authorProb.sum(axis=1)[:,np.newaxis],[1, 3])
y_Hat = uniformPriorWeight*(1/3.0) + (1.0-uniformPriorWeight)*authorProb_norm

# write a submission
submission = pd.read_csv('../input/sample_submission.csv')
submission.loc[:,['EAP', 'HPL', 'MWS']] = y_Hat
submission.to_csv("Markov_char_given_4charHistory.csv", index=False)
submission.head(10)

,id,EAP,HPL,MWS
0,id02310,0.030008,0.030024,0.939968
1,id24541,0.939964,0.030036,0.030000
2,id00134,0.219936,0.750057,0.030007
3,id27757,0.927792,0.042208,0.030000
4,id04081,0.624434,0.106144,0.269423
5,id27337,0.939336,0.030617,0.030047
6,id24265,0.829505,0.100380,0.070115
7,id25917,0.030005,0.030018,0.939976
8,id04951,0.939999,0.030000,0.030000
9,id14549,0.759949,0.104107,0.135943
